<a href="https://colab.research.google.com/github/Femagrinelli/solvimm/blob/main/Solvimm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

fileMoviesId = '1KxNNm0CWarlleFUQ3NcFvihaSm0nEtHu'
fileCustomersId = '1Oh3zGkP-AEkEmZ9Gym4Q59yz_1iR3oXS'

downloadMovies = drive.CreateFile({'id': fileMoviesId})
downloadMovies.GetContentFile('movies.csv')

downloadCustomers = drive.CreateFile({'id': fileCustomersId})
downloadCustomers.GetContentFile('customers_rating.csv')

dsMovies = pd.read_csv('movies.csv', header=None,sep=';')
dsCustomersRating = pd.read_csv('customers_rating.csv', sep=';').rename(
    columns={
        "Cust_Id": "customer_id",
        "Rating": "rating",
        "Date": "date",
        "Movie_Id": "movie_id"
    }
)




# TRATAMENTO DOS DADOS MOVIES.CSV

In [18]:
#O tratamento abaixo altera as colunas para facilitar a manipulação dos dados

dsMovies[['titulo','ano']] = dsMovies[1].str.split(',', expand=True) #-> separar o título do ano do filme pela vírgula.
dsMovies.pop(1)                                                      #-> remoção de uma coluna desnecessária
dsMovies.columns = ['movie_id', 'titulo', 'ano']                     #-> alteração dos nomes das colunas
dsMovies['ano'] = dsMovies['ano'].str.replace(')','')                #-> remoção do caracter parênteses
dsMovies['titulo'] = dsMovies['titulo'].str.replace('(','')          #-> remoção do caracter parênteses

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  import sys


# 1.1. Quantos filmes estão disponíveis no dataset?

In [20]:
#1.1 A quantidade de filmes disponíveis no dataset é feita simplesmente pegando o length do dataset dsMovies
print('O dataset possui {} filmes.'.format(len(dsMovies)))

O dataset possui 4499 filmes.


#1.2. Qual é o nome dos 5 filmes com melhor média de avaliação?

In [21]:
#1.2 Basta utilizar o groupby para agrupar por filme, e em seguida, a função de agregação MEAN para pegar a média de avaliação dos filmes.
#Por último, os valores foram ordenados de forma decrescente por rating médio
bestRatings = dsCustomersRating.groupby(['movie_id']).mean().sort_values('rating', ascending=False)

#O merge nesse caso, possibilita trazer o nome dos filmes fazendo o join no "movie_id", que é referenciado nos dois datasets.
pd.merge(bestRatings, dsMovies, on='movie_id').drop(columns = ['customer_id','ano']).head(5)


,movie_id,rating,titulo
0,3456,4.670989,Lost: Season 1
1,3033,4.586364,Ghost in the Shell: Stand Alone Complex: 2nd Gig
2,2102,4.581296,The Simpsons: Season 6
3,4238,4.554434,Inu-Yasha
4,13,4.552000,Lord of the Rings: The Return of the King: Ext...


#1.3. Quais os 5 anos com menos lançamentos de filmes?

In [26]:
#1.3 Utilização do groupby para agrupar por ano, e em seguida, a função de agregação COUNT para fazer a contagem total de filmes naquele ano.
fewYearReleases = dsMovies.groupby(['ano'])['ano'].count().reset_index(name='Movie count').sort_values(['Movie count'])
fewYearReleases.head(5)

,ano,Movie count
0,1915,1
2,1917,1
6,1922,1
9,1926,1
1,1916,2


#1.4. Quantos filmes que possuem avaliação maior ou igual a 4.7, considerando apenas os filmes avaliados na última data de avaliação do dataset?

In [23]:
#Primeiro, foi descoberto a ultima data de avaliação do dataset -> 31/12/2005
lastDate = dsCustomersRating.sort_values('date', ascending=False).iat[0,2]

#Aplicar uma variável para o filtro da avaliação
ratingFilter = 4.7

In [27]:
#Pegar o maior rating por movie_id na ultima data, e ordenar descrescente por esse maior rating e crescente pelo movie_id
lastDayHigherRatingPerMovie = dsCustomersRating.query('date == "{}"'.format(lastDate)).groupby(['movie_id'])['rating'].max().reset_index(name='higherRating').sort_values(['higherRating', 'movie_id'], ascending=[False, True])

#O tamanho do dataset foi alocado em uma variável. A query busca apenas os ratings >= que o filtro da avaliação criado na célular acima.
quantityMovies = len(lastDayHigherRatingPerMovie.query('higherRating >= {}'.format(ratingFilter)))
print('A quantidade de filmes que possuem avaliação maior ou igual a 4.7, considerando apenas os filmes avaliados na última data do dataset é {}.'.format(quantityMovies))

A quantidade de filmes que possuem avaliação maior ou igual a 4.7, considerando apenas os filmes avaliados na última data do dataset é 780.


#1.5. Dos filmes encontrados na questão anterior, quais são os 10 filmes com as piores notas e quais as notas?

In [28]:
#1.5 O dataframe "lastDayHigherRatingPerMovie" mostra a maior nota acima de 4.7 (feita na última data de avaliação do dataset) agrupadas por filme (movie_id),
#e está ordenada descrescente por avaliação e crescente pelo movie_id
#No cenário atual, como não tiveram notas como 4.7, 4.8 e 4.9 os filmes com as piores notas ainda foram os de nota 5, mas que estão listados ao final do dataframe.
#Supondo que houvesse uma nota 4.8, essa avaliação apareceria na última colocação por exemplo.

worstLastDayHigherRatingPerMovie = lastDayHigherRatingPerMovie.query('higherRating >= {}'.format(ratingFilter)).tail(10)

#Merge realizado para trazer os nomes dos filmes
pd.merge(worstLastDayHigherRatingPerMovie, dsMovies, on='movie_id').drop(columns = ['ano'])

,movie_id,higherRating,titulo
0,4454,5.0,To Have and Have Not
1,4468,5.0,Frasier: Season 6
2,4472,5.0,Love Actually
3,4479,5.0,Ordinary People
4,4485,5.0,House of Wax / The Mystery of the Wax Museum
5,4488,5.0,Wonder Boys
6,4489,5.0,Mysterious Island
7,4493,5.0,Ju-on: The Grudge
8,4495,5.0,Clifford: Happy Birthday Clifford / Puppy Love
9,4496,5.0,Farewell My Concubine


#1.6 Quais os id's dos 5 customer que mais avaliaram filmes e quantas avaliações cada um fez?


In [29]:
# Realizado um agrupamento a partir do customer_id, fazendo a contagem dos registros com o COUNT e a ordenação decrescente desse valor. Ao final, o head mostra apenas os 5 primeiros valores.
dsCustomersRating.groupby(['customer_id'])['customer_id'].count().reset_index(name='quantityCustomerRating').sort_values(['quantityCustomerRating'], ascending=False).head(5)

,customer_id,quantityCustomerRating
54307,305344,4467
69083,387418,4422
433441,2439493,4195
295928,1664010,4019
376450,2118461,3769
